In [ ]:
!pip install --upgrade yt-dlp

In [ ]:
!yt-dlp --version


In [ ]:
!apt update
!apt install ffmpeg
!ffmpeg -version

In [4]:
import os
import re
import tempfile
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import json
import librosa
import yt_dlp
from yt_dlp import YoutubeDL

In [5]:
# Функция для преобразования embed-ссылки в обычную
def convert_embed_to_watch_url(embed_url):
    """
    Преобразует ссылку формата embed в обычную ссылку на YouTube.
    """
    if "embed/" in embed_url:
        video_id = embed_url.split("embed/")[-1]
        return f"https://www.youtube.com/watch?v={video_id}"
    return embed_url

# Функция для проверки корректности ссылки на YouTube
def is_valid_youtube_url(url):
    """
    Проверяет, является ли ссылка корректной ссылкой на YouTube.
    """
    pattern = r"(?:v=|\/)([0-9A-Za-z_-]{11})"
    match = re.search(pattern, url)
    return match is not None

# Функция для загрузки аудио
def download_audio(url):
    try:
        regular_url = convert_embed_to_watch_url(url)
        if not regular_url:
            print(f"Некорректная ссылка на YouTube: {url}")
            return None

        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmp_file:
            tmp_file_path = 'output_audio.mp3'
            tmp_file.close()  # Закрываем файл перед использованием

        ydl_opts = {
          'format': 'bestaudio/best',
          'outtmpl': 'output_audio.%(ext)s',
          'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'mp3',
          'preferredquality': '192',
        }],
        'quiet': True,
        }


        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([regular_url])

        if not os.path.exists(tmp_file_path) or os.path.getsize(tmp_file_path) == 0:
            print(f"Файл аудио не был создан или пуст: {tmp_file_path}")
            os.remove(tmp_file_path)
            return None

        return tmp_file_path

    except Exception as e:
        print(f"Ошибка при загрузке аудио {regular_url}: {e}")
        return None

# Функция для извлечения Mel Spectrogram из аудио
def extract_mel_spectrogram(audio_path, n_mels=128, target_length=128):
    try:
        y, sr = librosa.load(audio_path, sr=None)

        # Проверка минимальной длины аудио
        if len(y) < target_length:
            print(f"Аудио слишком короткое: {audio_path}")
            return None

        mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
        mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

        if mel_spectrogram_db.shape[1] < target_length:
            padding = np.zeros((n_mels, target_length - mel_spectrogram_db.shape[1]))
            mel_spectrogram_db = np.hstack((mel_spectrogram_db, padding))
        else:
            mel_spectrogram_db = mel_spectrogram_db[:, :target_length]

        return mel_spectrogram_db

    except Exception as e:
        print(f"Ошибка при извлечении Mel Spectrogram: {e}")
        return None

# Определение модели классификации
class AudioClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(AudioClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Основной процесс
if __name__ == "__main__":
    # Параметры
    n_mels = 128  # Количество Mel-полос
    target_length = 128  # Фиксированная длина Mel Spectrogram
    json_path = "/content/COIN.json"
    max_audios_to_process = 100  # Ограничение количества аудио для обработки

    # Загрузка JSON-файла
    with open(json_path, "r") as f:
        data = json.load(f)

    database = data["database"]

    # Преобразование строковых категорий в числовые метки
    label_encoder = LabelEncoder()
    all_classes = [video_info["class"] for video_info in database.values()]
    label_encoder.fit(all_classes)
    num_classes = len(label_encoder.classes_)  # Количество уникальных классов

    # Сбор данных
    X = []
    y = []

    for video_id, video_info in database.items():
        if len(X) >= max_audios_to_process:
            break

        video_url = video_info["video_url"]
        video_class = video_info["class"]
        annotations = video_info["annotation"]

        if not is_valid_youtube_url(video_url):
            print(f"Некорректная ссылка на YouTube: {video_url}")
            continue

        audio_path = download_audio(video_url)
        if audio_path is None:
            print(f"Не удалось загрузить аудио по ссылке: {video_url}")
            continue

        for annotation in annotations:
            start_time, end_time = annotation["segment"]
            label = annotation["label"]

            mel_spectrogram = extract_mel_spectrogram(audio_path, n_mels=n_mels, target_length=target_length)
            if mel_spectrogram is None:
                print(f"Не удалось извлечь Mel Spectrogram из сегмента аудио: {video_url}")
                continue

            X.append(mel_spectrogram.flatten())
            y.append(video_class)

        os.remove(audio_path)

    if not X:
        raise ValueError("Нет данных для обучения. Проверьте аудиофайлы и их обработку.")

    X = torch.tensor(np.array(X), dtype=torch.float32)
    y = torch.tensor(label_encoder.transform(y), dtype=torch.long)

    print("Распределение меток:", Counter(y.numpy()))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    input_size = n_mels * target_length
    model = AudioClassifier(input_size=input_size, num_classes=num_classes)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)

    num_epochs = 15
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.numpy())
            y_pred.extend(predicted.numpy())

    y_true_labels = label_encoder.inverse_transform(y_true)
    y_pred_labels = label_encoder.inverse_transform(y_pred)

    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1-Score: {f1:.4f}')

ERROR: [youtube] ErhvSbJZ6mg: Video unavailable


Ошибка при загрузке аудио https://www.youtube.com/watch?v=ErhvSbJZ6mg: ERROR: [youtube] ErhvSbJZ6mg: Video unavailable
Не удалось загрузить аудио по ссылке: https://www.youtube.com/embed/ErhvSbJZ6mg


ERROR: [youtube] DvaCfRWskRE: Video unavailable


Ошибка при загрузке аудио https://www.youtube.com/watch?v=DvaCfRWskRE: ERROR: [youtube] DvaCfRWskRE: Video unavailable
Не удалось загрузить аудио по ссылке: https://www.youtube.com/embed/DvaCfRWskRE


ERROR: [youtube] cPEFskCrdhQ: Video unavailable


...

ERROR: [youtube] U-s6EYzqjmA: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Ошибка при загрузке аудио https://www.youtube.com/watch?v=U-s6EYzqjmA: ERROR: [youtube] U-s6EYzqjmA: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
Не удалось загрузить аудио по ссылке: https://www.youtube.com/embed/U-s6EYzqjmA


ERROR: [youtube] KcmZK7_oMIk: Video unavailable


Ошибка при загрузке аудио https://www.youtube.com/watch?v=KcmZK7_oMIk: ERROR: [youtube] KcmZK7_oMIk: Video unavailable
Не удалось загрузить аудио по ссылке: https://www.youtube.com/embed/KcmZK7_oMIk
Распределение меток: Counter({10: 10, 84: 7, 104: 6, 170: 6, 93: 5, 142: 5, 166: 5, 67: 5, 77: 4, 26: 4, 18: 4, 112: 3, 134: 3, 5: 3, 56: 3, 96: 3, 156: 3, 115: 3, 50: 3, 111: 3, 157: 3, 44: 2, 81: 2, 149: 2, 47: 2, 91: 2})
Epoch 1/15, Loss: 98.4587173461914
Epoch 2/15, Loss: 101.24079132080078
Epoch 3/15, Loss: 85.41593170166016
Epoch 4/15, Loss: 73.80070495605469
Epoch 5/15, Loss: 63.77655029296875
Epoch 6/15, Loss: 75.37283325195312
Epoch 7/15, Loss: 67.14269256591797
Epoch 8/15, Loss: 48.4975700378418
Epoch 9/15, Loss: 64.2437515258789
Epoch 10/15, Loss: 24.476423263549805
Epoch 11/15, Loss: 39.66626739501953
Epoch 12/15, Loss: 44.492942810058594
Epoch 13/15, Loss: 28.333984375
Epoch 14/15, Loss: 24.276256561279297
Epoch 15/15, Loss: 28.160505294799805
Precision: 0.2159
Recall: 0.2857
F

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
